In [1]:
import os
import sys
!pip install dacite
!pip install modal
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
    # sys.path.append(module_path)

In [2]:
from product_layer.report import ReportGenerator
from llms.gpt4 import GPT4, _GPT_4_MODEL_ID
from llms.perplexity import PerplexityLLM
from llms.baseten import BasetenModel, _BASETEN_MODEL_ID
from llms.modal import ModalModel
from product_layer.data_retrieval import get_team_scouting_input_data
from lib_db import db

/Users/raymondcano/Projects/scouting_report/server/llms/modal.py:46: PendingDeprecationError: 2024-04-29: The use of "Stub" has been deprecated in favor of "App". This is a pure name change with no other implications.
  stub = modal.Stub("vllm-scouting-report")


In [3]:
# establish a baseline model
baseline_model = GPT4(_GPT_4_MODEL_ID)


In [4]:
# create candidate models
# modal_mixtral = ModalModel()
gpt_4_turbo = GPT4('gpt-4-turbo')
gpt_35_turbo = GPT4('gpt-3.5-turbo-0125')
pplx_llama_3_small = PerplexityLLM('llama-3-sonar-small-32k-chat')
pplx_llama_3_large = PerplexityLLM('llama-3-sonar-large-32k-chat')
pplx_os_llama_3_70b = PerplexityLLM('llama-3-70b-instruct')

models = {
    gpt_4_turbo.get_name(): gpt_4_turbo,
    gpt_35_turbo.get_name(): gpt_35_turbo,
    pplx_llama_3_small.get_name(): pplx_llama_3_small,
    pplx_llama_3_large.get_name(): pplx_llama_3_large,
}

In [10]:
def add_to_reports(models):
    teams = db.get_teams_for_league('tdl')
    reports = []
    for t in teams: 
        report = db.get_most_recent_report(t.team_id)
        if not report or (report.created_at < '2024-05' and report.created_at < '2024-05-13'): 
            continue
        reports.append(report)
    print(len(reports))
    rg = ReportGenerator(models)
    results = []
    for idx, r in enumerate(reports):
        print(f'number: {idx}, {r.team_id}')
        result = rg.add_responses_to_report(None, r)
        db.write_team_scouting_report(result)
        results.append(result)
    return results


In [11]:
def update_with_pplx_llama():
    reports = add_to_reports([pplx_llama_3_large, pplx_llama_3_small])
    return reports
reports = update_with_pplx_llama()

17
number: 0, most-hated-d1
perplexity-llama-3-sonar-large-32k-chat
perplexity-llama-3-sonar-small-32k-chat
{"id": "0a3249cf-347d-47c5-b1b8-814e73268305", "model": "llama-3-sonar-small-32k-chat", "created": 1715639739, "usage": {"prompt_tokens": 15318, "completion_tokens": 431, "total_tokens": 15749}, "object": "chat.completion", "choices": [{"index": 0, "finish_reason": "stop", "message": {"role": "assistant", "content": "**Most Impactful Players**\nAlpha Bah, Cecil Tanner, Nayshon Kane\n\nAlpha Bah: 5.9 points, 5.1 rebounds, 1.6 assists, 0.371 true shooting percentage\nCecil Tanner: 8.4 points, 10.4 rebounds, 1.8 assists, 0.501 true shooting percentage\nNayshon Kane: 20.1 points, 5.7 rebounds, 3.0 assists, 0.570 true shooting percentage\n\n**Players We Can Take Advantage Of**\nSamson Rich, Mike Oxley\n\nSamson Rich: 4.9 points, 3.3 rebounds, 1.8 assists, 0.371 true shooting percentage, 2.9 turnovers, 1.8 fouls, 0.3 shooting percentage\nMike Oxley: 8.1 points, 2.4 rebounds, 2.4 assist

In [12]:
completed_team_ids = [x.team_id for x in reports]

In [8]:


teams = db.get_teams_for_league('tdl')
team_ids = [
    t.team_id for t in teams
]

import random
team_ids = random.sample(team_ids, 20)
print(len(team_ids))

20


In [10]:

completed_team_ids = []
for team_id in team_ids:
    try:
        rg = ReportGenerator(models=[baseline_model, gpt_4_turbo, gpt_35_turbo])
        data = get_team_scouting_input_data(team_id, games_back=5)
        reports = rg.generate_team_scouting_report(
            team_id=team_id,
            data=data,
        )
        db.write_team_scouting_report(reports)
        completed_team_ids.append(team_id)
    except Exception as e:
        print(f'failed for team {team_id} with {e}')



openai-gpt-4-0125-previewopenai-gpt-4-turbo

openai-gpt-3.5-turbo-0125
[Most Impactful Players]
- Henry Brooks:
  - Points: 13.6, Rebounds: 9.6, Assists: 1.9, True Shooting Percentage: 0.434
- Gio Porcelli:
  - Points: 11.3, Rebounds: 6.6, Assists: 8.4, True Shooting Percentage: 0.467
- Eli Baez:
  - Points: 15.1, Rebounds: 5.3, Assists: 3.0, True Shooting Percentage: 0.476

[Players We Can Take Advantage Of]
- Kirt Caesar:
  - Points: 3.1, Turnovers: 1.8, Personal Fouls: 1.4, Field Goal Percentage: 0.238
- Enoel Torres Jr.:
  - Points: 5.8, Turnovers: 0.9, Personal Fouls: 1.0, Field Goal Percentage: 0.188

[Team Overview]
- The team has been fairly consistent with their performance, having both close wins and losses.
- In closely contested games, there is a trend of balanced scoring across multiple players, with key players like Henry Brooks and Gio Porcelli stepping up.
- When the team wins by a significant margin, they tend to dominate in rebounding, especially with players like Hen

In [13]:
print(len(completed_team_ids))

17


In [63]:
all_reports = []
for team_id in completed_team_ids:
    all_reports.append(db.get_most_recent_report(team_id))

In [64]:
all_reports[0].responses.keys()

dict_keys(['openai-gpt-3.5-turbo-0125', 'openai-gpt-4-0125-preview', 'openai-gpt-4-turbo', 'perplexity-llama-3-sonar-small-32k-chat', 'perplexity-llama-3-sonar-large-32k-chat'])

In [65]:
print(len(all_reports))

17


In [66]:
eval_prompt = """
Repond with which answer to the prompt is better. 

[RUBRIC]
An ideal answer
- provides concrete actions to implement in a gameplan
- cites evidence from the provided data to back it's conclusions 
- emphasizes both threats and opportunities from the other team

[INSTRUCTIONS] 
Evaluate the two responses on a scale from 0 to 5, according to the [RUBRIC] above.

You should start your response by writing "[OBSERVATIONS]". Note the similarities and differences between the responses. 
Then, write out your answer by writing out "[ANSWER A SCORE]", then "[ANSWER B SCORE]". Lastly write "[WINNER]" followed by "A" or "B", 
which corresponds to the higher score.


[PROMPT]
{}

[Answer A]
{}

[Answer B]
{}
""".strip()


In [67]:
def parse_winner(answer):
    return answer.split('[WINNER]')[1].split()[0]

In [68]:
eval_results = []
system_prompt = 'you are an intelligent judge that follows the rubrics provided to evaluate responses to a prompt'
for report in all_reports:
    for model_name, llm_inf in report.responses.items():
        if model_name == baseline_model.get_name():
            continue
        baseline_response = report.responses[baseline_model.get_name()]
        formatted_prompt = eval_prompt.format(baseline_response.prompt, baseline_response.response, llm_inf.response)
        formatted_prompt_swapped = eval_prompt.format(baseline_response.prompt, llm_inf.response, baseline_response.response)
        resp = baseline_model.request(
            system_prompt,
            formatted_prompt,
            temperature=0.3,
        )
        resp_swapped = baseline_model.request(system_prompt, formatted_prompt_swapped, temperature=0.3)
        eval_res = dict(
            resp=resp,
            resp_swapped=resp_swapped,
            model_name=model_name,
            resp_winner=parse_winner(resp),
            resp_swapped_winner=parse_winner(resp_swapped),
            baseline=baseline_model.get_name(),
            report=report,
        )
        eval_results.append(eval_res)
    

In [69]:
from collections import defaultdict

win_map = defaultdict(int)
for eval_res in eval_results:
    challenger_wins = int(eval_res['resp_winner'] == 'B') + int(eval_res['resp_swapped_winner'] == 'A')
    baseline_wins = int(eval_res['resp_winner'] == 'A') + int(eval_res['resp_swapped_winner'] == 'B')
    win_map[eval_res['model_name']] += int(challenger_wins > baseline_wins)
    if challenger_wins == baseline_wins:
        win_map[eval_res['model_name']] += 0.5
win_rate = {k: v / len(all_reports) for k, v in win_map.items()}
win_rate

{'openai-gpt-3.5-turbo-0125': 0.20588235294117646,
 'openai-gpt-4-turbo': 0.38235294117647056,
 'perplexity-llama-3-sonar-small-32k-chat': 0.029411764705882353,
 'perplexity-llama-3-sonar-large-32k-chat': 0.058823529411764705}

In [70]:
import json
from dataclasses import asdict
for er in eval_results:
    er['report'] = asdict(er['report'])
with open('evals_v1.json', 'w') as f:
    eval_result = {
        'win_rates': win_rate,
        'eval_results': eval_results,
    }
    json.dump(eval_result, f)

In [84]:
def print_result(n):
    model_name = eval_results[n]['model_name'] 
    print(model_name)
    print(eval_results[n]['report']['responses'][model_name]['response'])
    print('baseline-------')
    print(eval_results[n]['report']['responses'][baseline_model.get_name()]['response'])
    print(eval_results[n]['resp'])
    print(eval_results[n]['resp_swapped'])


print_result(2)

perplexity-llama-3-sonar-small-32k-chat
**Most Impactful Players**
Alpha Bah, Cecil Tanner, Nayshon Kane

Alpha Bah: 5.9 points, 5.1 rebounds, 1.6 assists, 0.371 true shooting percentage
Cecil Tanner: 8.4 points, 10.4 rebounds, 1.8 assists, 0.501 true shooting percentage
Nayshon Kane: 20.1 points, 5.7 rebounds, 3.0 assists, 0.570 true shooting percentage

**Players We Can Take Advantage Of**
Samson Rich, Mike Oxley

Samson Rich: 4.9 points, 3.3 rebounds, 1.8 assists, 0.371 true shooting percentage, 2.9 turnovers, 1.8 fouls, 0.3 shooting percentage
Mike Oxley: 8.1 points, 2.4 rebounds, 2.4 assists, 0.423 true shooting percentage, 1.3 turnovers, 0.5 fouls, 0.358 shooting percentage

**Team Overview**
The team has been playing well recently, with a mix of close games and blowouts. In the games where they have lost or won closely, they tend to struggle with turnovers and fouls. In the games where they win by a lot, they tend to dominate the rebounding and scoring. They score most efficient

In [81]:
def correct_pplx_responses(filled_reports):
    models_to_fix = [pplx_llama_3_small, pplx_llama_3_large]
    for r in filled_reports:
        for m in models_to_fix:
            print(m)
            print(r.team_id)
            resp_st = r.responses[m.get_name()].response
            # print(type(resp_st))
            # print(resp_st)
            try:
                resp = json.loads(resp_st)
                choice = resp['choices'][0]['message']['content']
                r.responses[m.get_name()].response = choice
            except Exception as e:
                print(e)
            
            
        db.write_team_scouting_report(r)
# correct_pplx_responses(all_reports)
        

In [60]:
t_ids = [r.team_id for r in all_reports]

In [61]:
rep = db.get_most_recent_report(t_ids[0])

In [62]:
for k, v in rep.responses.items():
    print(k)
    print(v.response)

openai-gpt-3.5-turbo-0125
[Most Impactful Players]
- Nayshon Kane: Averaging 20.1 points, 5.7 rebounds, 3.0 assists, and a true shooting percentage of 57.0%
- Cecil Tanner: Averaging 8.4 points, 10.4 rebounds, 1.8 assists, and a true shooting percentage of 50.1%
- Branden Lawrence: Averaging 13.8 points, 4.4 rebounds, 2.8 assists, and a true shooting percentage of 59.4%

[Players We Can Take Advantage Of]
- Samson Rich: With weaknesses in turnovers (2.9 pg), personal fouls (1.8 pg), and shooting percentage (37.1% true shooting)
- Alpha Bah: Showing weaknesses in turnovers (1.0 pg), free throw percentage (23.1%), and point production (5.9 ppg)

[Team Overview]
- The team has been playing relatively well, with some close wins and losses indicating competitive matches.
- Patterns in close games show that Nayshon Kane and Cecil Tanner step up in scoring, but turnovers can be an issue in losses.
- In games with big wins, efficient scoring by Branden Lawrence and defensive presence by Cecil 